In [ ]:
import time
from random import random

import numpy as np
from pydrake.all import (
    DiagramBuilder,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    SolutionResult,
    Solve,
    StartMeshcat,
)
from pydrake.multibody import inverse_kinematics

from manipulation import running_as_notebook
from manipulation.exercises.trajectories.rrt_planner.robot import (
    ConfigurationSpace,
    Range,
)
from manipulation.exercises.trajectories.rrt_planner.rrt_planning import (
    Problem,
)
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import MakeManipulationStation

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
#Whole block here

In [ ]:
class RRT:
    """
    RRT Tree.
    """

    def __init__(self, root: TreeNode, cspace: ConfigurationSpace):
        self.root = root  # root TreeNode
        self.cspace = cspace  # robot.ConfigurationSpace
        self.size = 1  # int length of path
        self.max_recursion = 1000  # int length of longest possible path

    def add_configuration(self, parent_node, child_value):
        child_node = TreeNode(child_value, parent_node)
        parent_node.children.append(child_node)
        self.size += 1
        return child_node

    # Brute force nearest, handles general distance functions
    def nearest(self, configuration):
        """
        Finds the nearest node by distance to configuration in the
             configuration space.

        Args:
            configuration: tuple of floats representing a configuration of a
                robot

        Returns:
            closest: TreeNode. the closest node in the configuration space
                to configuration
            distance: float. distance from configuration to closest
        """
        assert self.cspace.valid_configuration(configuration)

        def recur(node, depth=0):
            closest, distance = node, self.cspace.distance(
                node.value, configuration
            )
            if depth < self.max_recursion:
                for child in node.children:
                    (child_closest, child_distance) = recur(child, depth + 1)
                    if child_distance < distance:
                        closest = child_closest
                        child_distance = child_distance
            return closest, distance

        return recur(self.root)[0]

In [ ]:
#Other code

In [ ]:
ik_solver = IKSolver()
q_goal, optimal = ik_solver.solve(T_WG_goal, q_guess=q_start)

In [ ]:
class RRT_tools:
    def __init__(self, problem):
        # rrt is a tree
        self.rrt_tree = RRT(TreeNode(problem.start), problem.cspace)
        problem.rrts = [self.rrt_tree]
        self.problem = problem

    def find_nearest_node_in_RRT_graph(self, q_sample):
        nearest_node = self.rrt_tree.nearest(q_sample)
        return nearest_node

    def sample_node_in_configuration_space(self):
        q_sample = self.problem.cspace.sample()
        return q_sample

    def calc_intermediate_qs_wo_collision(self, q_start, q_end):
        """create more samples by linear interpolation from q_start
        to q_end. Return all samples that are not in collision

        Example interpolated path:
        q_start, qa, qb, (Obstacle), qc , q_end
        returns >>> q_start, qa, qb
        """
        return self.problem.safe_path(q_start, q_end)

    def grow_rrt_tree(self, parent_node, q_sample):
        """
        add q_sample to the rrt tree as a child of the parent node
        returns the rrt tree node generated from q_sample
        """
        child_node = self.rrt_tree.add_configuration(parent_node, q_sample)
        return child_node

    def node_reaches_goal(self, node):
        return node.value == self.problem.goal

    def backup_path_from_node(self, node):
        path = [node.value]
        while node.parent is not None:
            node = node.parent
            path.append(node.value)
        path.reverse()
        return path

In [ ]:
def rrt_planning(problem, max_iterations=1000, prob_sample_q_goal=0.05):
    """
    Input:
        problem (IiwaProblem): instance of a utility class
        max_iterations: the maximum number of samples to be collected
        prob_sample_q_goal: the probability of sampling q_goal

    Output:
        path (list): [q_start, ...., q_goal].
                    Note q's are configurations, not RRT nodes
    """
    """ Input: q_start, q_goal, max_interation, prob_sample_goal
      Output: path

      G.init(q_start)
      for k = 1 to max_interation:
        q_sample ← Generate Random Configuration
        random number ← random()
        if random_number < prob_sample_goal:
            q_sample ← q_goal
        n_near ← Find the nearest node in the tree(q_sample)
        (q_1, q_2, ... q_N) ← Find intermediate q's from n_near to q_sample
        
        // iteratively add the new nodes to the tree to form a new edge
        last_node ← n_near
        for n = 1 to N:
            last_node ← Grow RRT tree (parent_node, q_{n}) 
        
        if last node reaches the goal:
            path ← backup the path recursively
            return path
        
      return None"""
    rrt_tools = RRT_tools(iiwa_problem)
    q_goal = problem.goal
    q_start = problem.start
    i = 0 
    while i < max_iterations:
        #generate random
        q_sample = rrt_tools.sample_node_in_configuration_space()
        rand_numb = random()
        if rand_numb < prob_sample_q_goal:
            q_sample = q_goal
        n_near = rrt_tools.find_nearest_node_in_RRT_graph(q_sample)
        intermediate_qs = rrt_tools.calc_intermediate_qs_wo_collision(n_near.value, q_sample)

        last_node = n_near
        for n in range(len(intermediate_qs)):
            last_node = rrt_tools.grow_rrt_tree(n_near, intermediate_qs[n])

        if rrt_tools.node_reaches_goal(last_node):
            path = rrt_tools.backup_path_from_node(last_node)
            return path
        i +=1
    return None